In [1]:
import torch
import torch.nn as nn

import functools
from functools import partial

from collections import defaultdict

from tqdm import tqdm

import numpy as np

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

from utils import build_model_and_tokenizer
from quant import quant, dequant
from auto_gptq import AutoGPTQForCausalLM

from transformers.pytorch_utils import Conv1D

model_name = 'facebook/opt-6.7b'
device = "cuda:1"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
testenc = tokenizer("\n\n".join(dataset["text"]), return_tensors="pt")

/home/jwjeong/anaconda3/envs/autogptq/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CUDA extension not installed.
CUDA extension not installed.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.21it/s]


activation 수집 후, q, k_proj을 통해 attn_map 생성

In [2]:
model.eval()
device = next(model.parameters()).device # next는 객체의 __next__ 호출, 다음 iter를 부름?
activation = {}

def stat_tensor(name, tensor):
    act_shape = tensor.shape
    hidden_dim = tensor.shape[-1]
    act_collect = tensor.detach().float().cpu()
    #tensor = tensor.view(-1, hidden_dim).abs().detach()
    if name in activation:
        activation[name].append(act_collect)
    else:
        activation[name] = list()
        activation[name].append(act_collect)


def stat_input_hook(m, x, y, name):
    if isinstance(x, tuple):
        x = x[0]
    stat_tensor(name, x)

name: model.decoder.layers.X.self_attn.k_proj

name: model.decoder.layers.X.fc1

In [3]:
hooks = []
for name, m in model.named_modules():
    if isinstance(m, nn.Linear) | isinstance(m, Conv1D):
        if name.split('.')[-1] == 'q_proj':
            hooks.append(
                m.register_forward_hook(
                    functools.partial(stat_input_hook, name=name))
            )

In [4]:
dataset = dataset.shuffle(seed=42)
dataset_list = []
for ii in range(len(dataset)):
    if dataset[ii]['text'] != '':
        dataset_list.append(dataset[ii])

In [5]:
num_samples = 128
seq_len = 512

for i in tqdm(range(num_samples)):
    input_ids = tokenizer(dataset_list[i]["text"], return_tensors="pt",
                            max_length=seq_len, truncation=True).input_ids.to(device)
    model(input_ids)

for h in hooks:
    h.remove()

100%|██████████| 128/128 [00:19<00:00,  6.44it/s]
